In [1]:
# Importando as libs necessárias

import pandas as pd
import numpy as np
import gensim
from gensim import corpora
from gensim.models import LdaModel, FastText, Word2Vec
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models, similarities
from gensim.matutils import hellinger
from gensim.matutils import sparse2full

In [ ]:
# Leitura dos dados do dataframe
#df = pd.read_csv('datasets/(processado-final)textos_tuitesPt_2020.csv.gz', names=['texto'])
df = pd.read_csv('datasets/(processado)textos_tuitesPt_2020_0.csv', names=['texto'])

# Elimina um valor flutuante que aparece no dataframe (por razões misteriosas)
# o algoritmo não aceita o valor flutuante, que precisa ser filtrado
df = df[df['texto'].apply(lambda x: isinstance(x, str))]
df['texto'].apply(type).value_counts()

df

In [3]:
# Extrair os textos dos documentos
docs = df['texto'].tolist()

# Tokenização dos documentos
docs_tokenizados = [doc.split() for doc in docs]

In [4]:
# Criar um dicionário a partir dos documentos tokenizados
dicionario = corpora.Dictionary(docs_tokenizados)

In [5]:
# Criar um corpus a partir do dicionário e dos documentos tokenizados
corpus = [dicionario.doc2bow(doc) for doc in docs_tokenizados]

In [6]:
# Treinar o modelo LDA
modelo_lda = gensim.models.LdaMulticore(corpus=corpus, id2word=dicionario, num_topics=10, passes=10, workers=4)

In [ ]:
# Imprimir os tópicos do modelo
for topico in modelo_lda.print_topics(num_words=20):
    print(topico)

In [ ]:
perplexidade = modelo_lda.log_perplexity(corpus)

print(f"A perplexidade do modelo LDA é {perplexidade}")

In [9]:
# Calcular a distância de tópicos
distance_matrix = similarities.MatrixSimilarity([modelo_lda.get_topic_terms(topicid) for topicid in range(modelo_lda.num_topics)])
hellinger_distance_matrix = distance_matrix.index
for i in range(modelo_lda.num_topics):
    for j in range(i+1, modelo_lda.num_topics):
        hellinger_distance_matrix[i][j] = hellinger(modelo_lda.get_topic_terms(i), modelo_lda.get_topic_terms(j))

# Imprimir a matriz de distância
print(hellinger_distance_matrix)

# Calcular a mediana da matriz de distância de Hellinger
median_distance = np.median(hellinger_distance_matrix)

print(f"A distância de tópicos média é {median_distance}")

In [11]:
# Obter as distribuições de palavras para cada tópico
topic_word_dists = modelo_lda.get_topics()

# Calcular a divergência KL-simétrica entre todos os pares de tópicos
num_topics = modelo_lda.num_topics
kl_divergence_matrix = np.zeros((num_topics, num_topics))
for i in range(num_topics):
    for j in range(i+1, num_topics):
        kl_divergence = 0.5 * (hellinger(sparse2full(modelo_lda.get_topic_terms(i), len(dicionario)), topic_word_dists[j]) +
                               hellinger(sparse2full(modelo_lda.get_topic_terms(j), len(dicionario)), topic_word_dists[i]))
        kl_divergence_matrix[i][j] = kl_divergence
        kl_divergence_matrix[j][i] = kl_divergence

# Imprimir a matriz de divergência KL-simétrica
print(kl_divergence_matrix)

In [13]:
# Calcular a mediana da matriz de distância de Hellinger
median_kl_divergence = np.median(kl_divergence_matrix)

print(f"A divergência KL média dos tópicos é {median_kl_divergence}")

A divergência KL média dos tópicos é 0.7109533816603362


In [ ]:
# Calcular a medida de coerência C_umass
coherence_model = CoherenceModel(model=modelo_lda, texts=docs_tokenizados, dictionary=dicionario, coherence='u_mass')
coherence_score = coherence_model.get_coherence()

# Imprimir a medida de coerência C_umass
print('A coerência C_umass dos tópicos é', coherence_score)

In [ ]:
# Calcular a medida de coerência C_w2v
coherence_model = CoherenceModel(model=modelo_lda, texts=docs_tokenizados, dictionary=dicionario, coherence='c_w2v')
coherence_score = coherence_model.get_coherence()

# Imprimir a medida de coerência C_w2v
print('A coerência C_w2v dos tópicos é',coherence_score)